In [1]:
import pandas as pd
import json
import os
import ast 

import numpy as np
from tqdm import tqdm
import chardet
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import textblob
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import math
from nltk import bigrams,trigrams,ngrams
from collections import Counter
from sklearn.metrics import confusion_matrix
from nltk import everygrams, word_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_confusion_matrix

# Deal with business data

In [2]:
business = pd.read_json('./business.json',lines = True)
CA_business = business[business['state'] == 'CA']
CA_business = CA_business.reset_index()

In [3]:
for i in range(len(CA_business)):
    cur_attr = CA_business['attributes'][i]
    if bool(cur_attr) == False:
        continue
    for key in cur_attr.keys():
        CA_business.loc[i,key] = cur_attr[key]


def extract_info (df, feature):
    for i in range(len(df)):
        try:
            cur_attr = df[feature][i]
            cur_attr = ast.literal_eval(cur_attr)
            for key in cur_attr.keys():
                df.loc[i,key] = cur_attr[key]
        except: 
            continue

            
def extract_info_hours (df, feature):
    for i in range(len(df)):
        try:
            cur_attr = df[feature][i]
            #cur_attr = ast.literal_eval(cur_attr)
            for key in cur_attr.keys():
                df.loc[i,key] = cur_attr[key]
        except: 
            continue
            
extract_info(CA_business,'BusinessParking')    
extract_info(CA_business,'GoodForMeal')     
extract_info(CA_business,'Ambience')     
extract_info_hours(CA_business,'hours')     
     
        
CA_business = CA_business.drop(columns=['attributes','BusinessParking','GoodForMeal','Ambience'])
CA_business = CA_business.rename({'stars':'average_stars'},axis = 1)

In [4]:
Asian_keyword = ['Chinese','Korean','Japanese','Asian','Sushi']
Asian_list = []
for i in range(len(CA_business)):
    cur_cat = CA_business.loc[i,'categories']
    try: 
        cur_cat = cur_cat.split(',')
        if any(item in cur_cat for item in Asian_keyword):
            Asian_list.append(CA_business.loc[i,'name'])
    except: continue
        
CA_Asian_food = CA_business[CA_business.name.str.contains('|'.join(Asian_list))]
#CA_Asian_food = CA_Asian_food.reset_index(drop=True)

# Turn Monday-Sunday working hour to length

In [5]:
def convert_hour(time):
    result = 0
    if len(time) == 5:
        if time[-2] == '00':
            result = int(time[:-3])
        elif int(time[-2]) > 15:
            result = int(time[:-3])+1
        else:
            result = int(time[:-3])
    elif len(time) == 4:
        if time[-1] == '0':
            result = int(time[:-2])
        elif int(time[-1]) > 15:
            result = int(time[:-2])+1
        else:
            result = int(time[:-2])
    else:
        if time[-1] == '0':
            result = int(time[:-2])
        elif int(time[-1]) > 15:
            result = int(time[:-2])+1
        else:
            result = int(time[:-2])
    return result
            

def calculate_total_hour(feature, new_feature):
    for i in range(len(CA_Asian_food)):
        try:
            cur_schedule =  CA_Asian_food.loc[i,feature]
            ls = cur_schedule.split('-')
            if ls[0] == ls[1]:
                CA_Asian_food.loc[i,new_feature] = 24
            else:
                work_hour = convert_hour(ls[1])-convert_hour(ls[0])
                CA_Asian_food.loc[i,new_feature] =work_hour
        except: 
            continue
        

calculate_total_hour('Monday','Monday_working_length')
calculate_total_hour('Tuesday','Tuesday_working_length')
calculate_total_hour('Wednesday','Wednesday_working_length')
calculate_total_hour('Thursday','Thursday_working_length')
calculate_total_hour('Friday','Friday_working_length')
calculate_total_hour('Saturday','Saturday_working_length')
calculate_total_hour('Sunday','Sunday_working_length')

CA_Asian_food = CA_Asian_food.drop(columns=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']).reset_index(drop = True)

# Read and combine reivew with business

In [10]:
review = pd.read_csv('./review.csv')
review = review.rename({'stars':'comment_star'},axis = 1)
review.head()

,review_id,user_id,business_id,comment_star,date,text
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,2014-02-05 20:30:30,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,2015-01-04 00:01:03,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,2017-01-14 20:54:15,Cute interior and owner (?) gave us tour of up...


In [11]:
frames = [CA_Asian_food, review]
CA_Asian_business_review = pd.merge(review, CA_Asian_food,left_on='business_id',right_on = 'business_id', how = 'right' ).reset_index(drop=True)
CA_Asian_business_review.to_csv('CA_Asian_business_review.csv',index = False)